In [20]:
import re
def clean_text(rgx_list, text):
    new_text = text
    for rgx_match in rgx_list:
        new_text = re.sub(rgx_match, '', new_text)
    return new_text

In [44]:
#Initialization for HateSpeechRecognition
import scipy as sp
import numpy as np
import pandas as pd
import sklearn
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt')

stopwords_list = stopwords.words('english')
#other_exclusions = ["#ff", "ff", "rt","!",":","...",".","-","&","?"]
#stopwords_list.extend(other_exclusions)

dataset = pd.read_csv("dataset/labeled_data.csv")

tweets = dataset.tweet

ps = nltk.stem.PorterStemmer()

#ps.stem(dataset)
filtered_tweets=[];
for tweet in tweets:
    tweet = clean_text([".*@.*:","&#*\w*","@[\w\-]+","[^\w\s]","http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+][!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+"],tweet)
    tweet = tweet.lower()
    word_tokens = word_tokenize(tweet)
    filtered_tweets.append([word for word in word_tokens if not word in stopwords_list])

print(filtered_tweets[0:20])



[nltk_data] Downloading package stopwords to /home/mufa/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/mufa/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[['woman', 'shouldnt', 'complain', 'cleaning', 'house', 'man', 'always', 'take', 'trash'], ['boy', 'dats', 'coldtyga', 'dwn', 'bad', 'cuffin', 'dat', 'hoe', '1st', 'place'], ['ever', 'fuck', 'bitch', 'start', 'cry', 'confused', 'shit'], ['look', 'like', 'tranny'], ['shit', 'hear', 'might', 'true', 'might', 'faker', 'bitch', 'told', 'ya'], ['shit', 'blows', 'meclaim', 'faithful', 'somebody', 'still', 'fucking', 'hoes'], ['sit', 'hate', 'another', 'bitch', 'got', 'much', 'shit', 'going'], ['cause', 'im', 'tired', 'big', 'bitches', 'coming', 'us', 'skinny', 'girls'], ['might', 'get', 'ya', 'bitch', 'back', 'thats'], ['fighting', 'mariam', 'bitch'], ['keeks', 'bitch', 'curves', 'everyone', 'lol', 'walked', 'conversation', 'like', 'smh'], ['murda', 'gang'